In [3]:
import dagshub
dagshub.init(repo_owner='AMR-ITH', repo_name='yt-comment-analyzer', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4f305c81-34a9-4761-842c-41c55d41a4ad&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=3a9f005cba86e8e68c7990144cba795a9b7092449170cbd4f7a65c63a6ad4f97




Accessing as AMR-ITH

Initialized MLflow to track repo "AMR-ITH/yt-comment-analyzer"

Repository AMR-ITH/yt-comment-analyzer initialized!

In [8]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/6372c8dc1c894e3290997ce5e2b55158', creation_time=1751625225992, experiment_id='4', last_update_time=1751625225992, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna
from imblearn.under_sampling import RandomUnderSampler

In [9]:
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [11]:
# Step 1: Clean data
df = df.dropna(subset=['category'])
y = df['category'].map({-1: 2, 0: 0, 1: 1})
X_raw = df['clean_comment']

# Step 2: Train-test split BEFORE vectorization or resampling
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Vectorization only on training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=3000)
X_train_vect = vectorizer.fit_transform(X_train_raw)
X_test_vect = vectorizer.transform(X_test_raw)  # Note: transform only!

# Step 4: Undersampling (only on training data)
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_vect, y_train)

# # Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42, stratify=y_train_resampled)




# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_RandomUnderSampler_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)



# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for LightGBM
run_optuna_experiment()


[I 2025-07-11 16:43:57,904] A new study created in memory with name: no-name-2b5c5202-f1b9-4a69-87cb-a5233ab63c8a
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.234656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 64705
[LightGBM] [Info] Number of data points in the train set: 15835, number of used features: 2385
[LightGBM] [Info] Start training from score -1.098486
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-07-11 16:44:04,027] Trial 0 finished with value: 0.7956554685526648 and parameters: {'n_estimators': 269, 'learning_rate': 0.09177880551980942, 'max_depth': 4}. Best is trial 0 with value: 0.7956554685526648.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.396194 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 64705
[LightGBM] [Info] Number of data points in the train set: 15835, number of used features: 2385
[LightGBM] [Info] Start training from score -1.098486
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-07-11 16:44:06,923] Trial 1 finished with value: 0.5145238696640566 and parameters: {'n_estimators': 96, 'learning_rate': 0.000630701918262064, 'max_depth': 3}. Best is trial 0 with value: 0.7956554685526648.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.246767 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 64705
[LightGBM] [Info] Number of data points in the train set: 15835, number of used features: 2385
[LightGBM] [Info] Start training from score -1.098486
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-07-11 16:47:54,676] Trial 29 finished with value: 0.7903510987623137 and parameters: {'n_estimators': 177, 'learning_rate': 0.06197482163267644, 'max_depth': 9}. Best is trial 18 with value: 0.8024753725688305.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.448733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 64705
[LightGBM] [Info] Number of data points in the train set: 15835, number of used features: 2385
[LightGBM] [Info] Start training from score -1.098486
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


🏃 View run LightGBM_RandomUnderSampler_TFIDF_Trigrams at: https://dagshub.com/AMR-ITH/yt-comment-analyzer.mlflow/#/experiments/4/runs/b566b25f0b8f418d8e6ea9bc023b88d5
🧪 View experiment at: https://dagshub.com/AMR-ITH/yt-comment-analyzer.mlflow/#/experiments/4
